In [88]:
import happybase
conn=happybase.Connection('localhost',8080)
table=conn.table('users')

In [2]:
def get_data():
    import random
    name=input('Name: ')
    From=input('From: ')
    education=input('Education: ')
    profession=input('Profession: ')
    location=input('Current location: ')
    Birthday=input('Birthday')
    
    
    
    #id_u='R1'
    
    while True:
        num=random.randint(1, 10001)
        id_u='R'+name.replace(' ','_')+'_'+Birthday[:2]+'_'+str(num)
        #print(id_u)
    
    
        try:
            row=table.row(id_u)
            #print("b")
        except:
            row={}
        print(row)
        if len(row)==0:
            #print("c")
            break
    #id_b=''.join(format(ord(i), 'b') for i in id_u)
    #name_b=''.join(format(ord(i), 'b') for i in name)
    #from_b=''.join(format(ord(i), 'b') for i in From)
    #edu_b=''.join(format(ord(i), 'b') for i in education)
    #pro_b=''.join(format(ord(i), 'b') for i in profession)
    #loc_b=''.join(format(ord(i), 'b') for i in location)
    #bir_b=''.join(format(ord(i), 'b') for i in Birthday)
    
    
    
    #print("a")
    
    table.put(id_u, {b'cf_user_1:name': name,b'cf_user_1:from': From,b'cf_user_1:Education': education,b'cf_user_1:Profession': profession,b'cf_user_1:location': location,b'cf_user_1:birthday': Birthday}) 
    #table.put(b'R2',{b'cf_user_1:c1':b'v1',b'cf_user_1:c2':b'v2'})
        
        
        
        
    
    

In [89]:
#table.put('R3',{b'cf_user_1:c1':'v1',b'cf_user_1:c2':'v2'})
row=table.row('R1')
print(row)

{b'cf_user_1:id': b'1'}


In [90]:
import MySQLdb
import mysql.connector
conn= mysql.connector.connect(host="localhost",user="abhijit",passwd="1234",db="users" )


In [91]:
import pymongo
from pymongo import MongoClient
client = MongoClient()
client = MongoClient('localhost', 27017)

In [92]:
db_mongo = client['user_connections']
m_user_recs= db_mongo['user_records']

In [84]:
m_user_recs

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'user_connections'), 'user_records')

In [29]:
m_user_recs.remove()

/home/abhijit/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  """Entry point for launching an IPython kernel.


{'n': 3, 'ok': 1.0}

In [19]:
data={
	'c_id':'abhi',
	'connections':['123','myac.a','abhir'],
	'posts':[]}

In [ ]:
d={'ar':1,'br':4,'cr':3,'dr':5,'er':2}

In [ ]:
s = sorted(d, key=lambda x: x[1], reverse=True)

In [ ]:
s

In [ ]:
data

In [20]:
m_user_recs.insert(data)

/home/abhijit/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  """Entry point for launching an IPython kernel.


ObjectId('5f942323f9206b6c0904ef00')

In [30]:
fr=m_user_recs.find()
for r in fr:
    print(r)

In [93]:
id1='234'
email='abc@gmail.com'
passwd='1234'
mycursor = conn.cursor()
#mycursor.execute('INSERT INTO user_details (id,email,password) VALUES("%s","%s","%s")'%(id1,email,passwd))
#conn.commit()

In [94]:
from flask import Flask,request,render_template,redirect,flash
import pickle
from werkzeug import secure_filename
import numpy as np
import flask
import os


#Settings

UPLOAD_FOLDER = 'uploads'

app = Flask(__name__, template_folder='templates')
#app.secret_key = "secret key"
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

#Response Template

def sendResponse(responseObj):
    response = flask.jsonify(responseObj)
    response.headers.add('Access-Control-Allow-Origin', '*')
    response.headers.add('Access-Control-Allow-Methods', 'GET')
    response.headers.add('Access-Control-Allow-Headers', 'accept,content-type,Origin,X-Requested-With,Content-Type,access_token,Accept,Authorization,source')
    response.headers.add('Access-Control-Allow-Credentials', True)
    return response

#Signup

@app.route('/signup')
def signup():
    return render_template('get.html')

@app.route('/signup_details',methods=['POST'])
def signup_details():
    if request.method=='POST':
        email=request.form.get('Email')
        passwd=request.form.get('Passwd')
        con_passwd=request.form.get('Con_Passwd')
    
        if email=='' or passwd=='' or con_passwd=='':
            #flash('fields empty')
            return redirect(request.url)
    
        if passwd!=con_passwd:
            #flash('Password do not match')
            return redirect(request.url)
        query="""SELECT * FROM user_details WHERE email='%s'"""%(email)
        mycursor = conn.cursor()
        mycursor.execute(query)
        dup=mycursor.fetchall()
        i=mycursor.rowcount
    
        if i!=0:
            #flash('Emails exist')
            return redirect(request.url)
        mycursor = conn.cursor()
        id1=email.split('@')[0]
        print(id1)
        mycursor = conn.cursor()
        mycursor.execute('INSERT INTO user_details (id,email,password) VALUES("%s","%s","%s")'%(id1,email,passwd))
        conn.commit()
        print("A")
        return render_template('signup.html',name=id1)
    else:
        return redirect(request.url)
    
#Fill Details    

@app.route('/personal_info',methods=['POST'])
def personal_info():

    fields=['usr_name','Name','From','Education','Profession','location','Birthday']
    
    record=[]
    if request.method=='POST':
        for i in range(len(fields)):
            if request.form.get(fields[i])=="":
                #flash('fields missing')
                return redirect(request.url)
            record.append(request.form.get(fields[i]))
        if 'file' not in request.files:
            #flash('No file part')
            return redirect(request.url)
        file = request.files['file']
        if file.filename == '':
            #flash('No file selected for uploading')
            return redirect(request.url)
        if file:
            filename = secure_filename(file.filename)
            file.save(os.path.join(app.config['UPLOAD_FOLDER'],filename))
            na=os.path.join(app.config['UPLOAD_FOLDER'],filename)
        record.append(na)
        print(record)
        #print()
        while True:
            id_u='R'+record[0]
            #print(id_u)
    
    
            try:
                row=table.row(id_u)
                #print("b")
            except:
                row={}
            print(row)
            if len(row)==0:
                #print("c")
                break
        
        table.put(id_u, {b'cf_user_1:name': record[1],b'cf_user_1:from': record[2],b'cf_user_1:Education': record[3],b'cf_user_1:Profession': record[4],b'cf_user_1:location': record[5],b'cf_user_1:birthday': record[6],b'cf_user_1:profile_pic': record[7]})
        return render_template('success.html')
    else:
        return redirect(request.url)
        
#Log in    

@app.route('/login',methods=['POST'])
def login():
    if request.method=='POST':
        return render_template('login.html')
    else:
        return redirect(request.url)
@app.route('/log')
def log():
    return render_template('login.html')

@app.route('/login_det',methods=['POST'])
def login_det():
    if request.method=='POST':
        email=request.form.get('Email')
        passwd=request.form.get('Passwd')
        query="""SELECT password FROM user_details WHERE email='%s'"""%(email)
        mycursor = conn.cursor()
        mycursor.execute(query)
        
        record=mycursor.fetchall()
        
        if record==0:
            #flash('The email is not registered.Please register')
            return render_template('get.html')
        
        else:
            print(record[0][0])
            if passwd==record[0][0]:
                query="""SELECT id FROM user_details WHERE email='%s'"""%(email)
                mycursor = conn.cursor()
                mycursor.execute(query)
                id1=mycursor.fetchall()
                print(id1[0][0])
                return render_template('succ_login.html',name=id1[0][0])
            else:
                #flash('Wrong password. Please try again')
                return redirect(request.url)
    else:
        
        return redirect(request.url)

#home    
    
@app.route('/cont',methods=['POST'])
def cont():
    if request.method=='POST':
        id1=request.form.get('usr_name')
        return render_template('home.html',name=id1)
    else:
        return redirect(request.url)

#add friends    
    
@app.route('/add_fr',methods=['POST'])
def add_fr():
    if request.method=='POST':
        id1=request.form.get('usr_name')
        id_k='R'+id1
        row_u=table.row(id_k)
        #print(row_u)
        l_u=row_u[b'cf_user_1:location']
        f_u=row_u[b'cf_user_1:from']
        e_u=row_u[b'cf_user_1:Education']
        #print(l_u)
        try:
            recs=m_user_recs.find({"c_id":id1})
        except:
            recs=[]
        #print(recs)
        #print("A")
        if recs.count()==0:
            return render_template('first_add.html',name=id1)
        else:
            recs=m_user_recs.find({"c_id":id1})
            #print(recs)
            #print("V") 
            for r in recs: 
                frnds=r['connections']
            #print(frnds)
            may_be=[]
            for f in frnds:
                recs=m_user_recs.find({"c_id":f})
                temp=[]
                for i in recs:
                    temp=i['connections']
                for t in temp:
                    if t not in may_be:
                        may_be.append(t)
            #print("C")
            #print(may_be)
            pred={}
            loc=[]
            fr=[]    
            edu=[]
            for m in may_be:
                u='R'+m
                row_f=table.row(u)
                #print(row_f)
                loc.append(row_f[b'cf_user_1:location'])
                fr.append(row_f[b'cf_user_1:from'])
                edu.append(row_f[b'cf_user_1:Education'])
            for i in range(len(edu)):
                score=0
                if l_u==loc[i]:
                    score+=1
                if f_u==fr[i]:
                    score+=1
                if e_u==edu[i]:
                    score+=1
                pred[may_be[i]]=score
            pred_s = sorted(pred, key=lambda x: x[1], reverse=True)
            for f in frnds:
                pred_s.remove(f)               
                
            return render_template('add_friend.html',name=id1,pred_1=pred_s[0],pred_2=pred_s[1],pred_3=pred_s[2],pred_4=pred_s[3],pred_5=pred_s[4])            
    else:
        return redirect(request.url)
    
#Search Friends

@app.route('/search',methods=['POST'])
def search():
    if request.method=='POST':
        id1=request.form.get('usr_name')
        srch=request.form.get('search')
        
        search_id='R'+srch
        row=table.row(search_id)
        if len(row)==0:
            #flash('Oops the person is not present')
            return render_template('home.html')
        
        name=row[b'cf_user_1:name']
        address=row[b'cf_user_1:location']
        edu=row[b'cf_user_1:Education']
        
        return render_template('search_obtain.html',nam=id1,na=name,ad=address,ed=edu,sr=srch)
    
    else:
        return redirect(request.url)    

#Connect

@app.route('/connect',methods=['POST'])
def connect():
    if request.method=='POST':
        id1=request.form.get('usr_name')
        con=request.form.get('con_name')
        
        
        
        
    

#Create posts

@app.route('/create_p',methods=['POST'])
def create_p():
    if request.method=='POST':
        id1=request.form.get('usr_name')
        return render_template('create.html',name=id1)
    else:
        return redirect(request.url)

#back

@app.route('/back',methods=['POST'])
def back():
    if request.method=='POST':
        id1=request.form.get('usr_name')
        return render_template('home.html',name=id1)
    else:
        return redirect(request.url)
    
#get feed
    
@app.route('/feed_v',methods=['POST'])
def feed_v():
    if request.method=='POST':
        id1=request.form.get('usr_name')
        return render_template('feed.html',name=id1)
    else:
        return redirect(request.url)

#view friends    
    
@app.route('/view_fr',methods=['POST'])
def view_fr():
    if request.method=='POST':
        id1=request.form.get('usr_name')
        return render_template('view.html',name=id1)
    else:
        return redirect(request.url)

        
if __name__ == "__main__":
    app.run()
    
    

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [25/Oct/2020 00:00:10] "POST /search HTTP/1.1" 200 -
127.0.0.1 - - [25/Oct/2020 00:00:21] "POST /connect HTTP/1.1" 404 -
